<a href="https://colab.research.google.com/github/MarshaKamsteeg/TM10007_Machine_Learning_Group_Assignment/blob/main/tm10007_group_9_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TM10007 Assignment

In [5]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [8]:
# Import Data
from adni.load_data import load_data


data = load_data()


print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')



                     hf_energy  hf_entropy  hf_kurtosis    hf_max   hf_mean  \
ID                                                                            
0_002_S_0413_bl_0  5403.229408    4.679203     2.392463  2.040475  1.375697   
0_002_S_0559_bl_0   922.459760    4.772376     1.709731  1.437500  0.863074   
0_002_S_0619_bl_0  1674.765276    5.369472    -1.051024  1.691528  0.744530   
0_002_S_0685_bl_0  3443.797200    4.613089     3.404878  1.775424  1.236849   
0_002_S_0816_bl_0   914.469846    4.647179     3.574511  1.453932  0.949651   
...                        ...         ...          ...       ...       ...   
0_941_S_4292_bl_0  1503.564446    4.490768     2.368284  1.724321  1.094387   
0_941_S_4365_bl_0  1087.136508    4.929075     0.284622  1.582494  0.859611   
0_941_S_4376_bl_0  1963.323949    4.619950     2.012379  1.794024  1.167033   
0_941_S_5124_bl_0  2392.983035    5.139234    -0.655371  2.023878  1.051669   
0_941_S_5193_bl_0  2350.828850    4.568731     1.704